In [1]:
import psycopg2
import random
import string
from tqdm import tqdm

In [2]:
# Database Information
DATABASE_NAME = "users_test1"
DATABASE_USERNAME = ""
DATABASE_PASSWORD = ""
DATABASE_HOST = "localhost"
DATABASE_POST = "5432"

# NOTE: Please ensure DATABASE_NAME (users_test1 in this case) 
# database exists in the Postgres server

# Establishing the connection
conn = psycopg2.connect(
    database=DATABASE_NAME,
    user=DATABASE_USERNAME,
    password=DATABASE_PASSWORD,
    host=DATABASE_HOST,
    port=DATABASE_POST,
)

# Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [3]:
# Droping EMPLOYEE table if already exists.
cursor.execute("DROP TABLE IF EXISTS EMPLOYEE")

# Creating table as per requirement
sql = """CREATE TABLE EMPLOYEE(
   FIRST_NAME CHAR(20) NOT NULL,
   LAST_NAME CHAR(20),
   AGE INT,
   SEX CHAR(1),
   INCOME FLOAT
)"""

cursor.execute(sql)
conn.commit()

print("Table created successfully")

Table created successfully


In [4]:
# Preparing SQL queries to INSERT a record into the database.
for i in tqdm(range(0, 500000)):
    sql = f"""
        INSERT INTO EMPLOYEE(
            FIRST_NAME, 
            LAST_NAME, 
            AGE, 
            SEX,
            INCOME
        ) VALUES (
            '{''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase, k=5))}', 
            '{''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase, k=5))}', 
            {random.randint(22, 58)}, 
            '{random.choice(['F','M'])}', 
            {random.randint(100000, 1000000)}
        )
    """
    cursor.execute(sql)

# Commit your changes in the database
conn.commit()

print("Records inserted")

100%|██████████| 500000/500000 [00:22<00:00, 22480.35it/s]

Records inserted


# Testing Index on age column

In [21]:
# Querying without index
sql = """
    EXPLAIN ANALYZE SELECT * FROM EMPLOYEE WHERE age = 34;
"""
cursor.execute(sql)
cursor.fetchall()

[('Gather  (cost=1000.00..10681.17 rows=13950 width=56) (actual time=0.222..28.613 rows=13465 loops=1)',),
 ('  Workers Planned: 2',),
 ('  Workers Launched: 2',),
 ('  ->  Parallel Seq Scan on employee  (cost=0.00..8286.17 rows=5812 width=56) (actual time=0.019..18.403 rows=4488 loops=3)',),
 ('        Filter: (age = 34)',),
 ('        Rows Removed by Filter: 162178',),
 ('Planning Time: 0.057 ms',),
 ('Execution Time: 29.083 ms',)]

In [22]:
# Create an index on age column
sql = """
    CREATE INDEX IDX_EMPLOYEE_AGE 
    ON EMPLOYEE(age);
"""
cursor.execute(sql)
conn.commit()

In [27]:
# Querying with index
sql = """
    EXPLAIN ANALYZE SELECT * FROM EMPLOYEE WHERE age = 34;
"""
cursor.execute(sql)
cursor.fetchall()

[('Bitmap Heap Scan on employee  (cost=264.54..6120.91 rows=13950 width=56) (actual time=1.123..5.512 rows=13465 loops=1)',),
 ('  Recheck Cond: (age = 34)',),
 ('  Heap Blocks: exact=5161',),
 ('  ->  Bitmap Index Scan on idx_employee_age  (cost=0.00..261.05 rows=13950 width=0) (actual time=0.638..0.638 rows=13465 loops=1)',),
 ('        Index Cond: (age = 34)',),
 ('Planning Time: 0.048 ms',),
 ('Execution Time: 5.786 ms',)]